In [1]:
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets
import pandas as pd
from datetime import timedelta
import re

In [2]:

lac_2022_df = pd.read_csv('data/lac_2022_pbp.csv')
lac_2022_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback."
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...


In [3]:
roster_df = pd.read_csv('data/lac_2022_roster.csv')
roster_list=roster_df['roster'].tolist()
print(roster_list)
#roster_df.head()

['Nasir Adderley', 'Keenan Allen', 'Jeremiah Attaochu', 'Zack Bailey', 'Michael Bandy', 'Taylor Bertolet', 'Joey Bosa', 'Bryce Callahan', 'DeAndre Carter', 'Will Clapp', 'Christian Covington', 'Chase Daniel', 'Michael Davis', 'Tyeler Davison', 'Cameron Dicker', 'Austin Ekeler', 'Gerald Everett', 'Breiden Fehoko', 'Matt Feiler', 'Morgan Fox', 'Joe Gaziano', 'Alohi Gilman', 'Jalen Guyton', 'Kemon Hall', 'Josh Harris', 'Justin Herbert', 'Christopher Hinton', 'Dustin Hopkins', 'Zander Horvath', 'J.C. Jackson', 'Brenden Jaimes', 'Derwin James', 'Austin Johnson', 'Zion Johnson', 'Sebastian Joseph-Day', 'Joshua Kelley', 'Raheem Layne', 'Deane Leonard', 'Corey Linsley', 'Khalil Mack', "Tre' McKitty", 'Sony Michel', 'David Moa', 'Jason Moore', 'Kenneth Murray', 'Nick Niemann', 'Storm Norton', 'Amen Ogbongbemiga', 'Otito Ogbonnia', 'Josh Palmer', 'Donald Parham', 'Trey Pipkins', 'Troy Reeder', 'Richard Rodgers', 'Larry Rountree', 'Chris Rumph', 'Jamaree Salyer', 'Asante Samuel Jr.', 'Foster Sare

In [4]:
def possession_check(sentence, roster_list,team_1_abrv,):
    # Split the sentence into words
    words = sentence.split()
    # each detail begins with the name of a player on the 
    name = ' '.join(words[:2])
    # Check if the name is on thhe roster list, if not then it is the other
    if name in roster_list:
        return team_1_abrv
    else:
        return #"OPP"

# Define your list of words to check against
# Apply the function to create a new column called "possession_team"






In [5]:
lac_2022_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3063 entries, 0 to 3062
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   week       3063 non-null   int64  
 1   Quarter    3062 non-null   object 
 2   Time       3062 non-null   object 
 3   Down       2660 non-null   float64
 4   ToGo       2660 non-null   float64
 5   Location   2941 non-null   object 
 6   OPP        3063 non-null   object 
 7   OPP Score  2940 non-null   float64
 8   LAC Score  2940 non-null   float64
 9   Detail     3063 non-null   object 
dtypes: float64(4), int64(1), object(5)
memory usage: 239.4+ KB


In [6]:
lac_2022_df['possession_team'] = lac_2022_df['Detail'].apply(lambda x: possession_check(x, roster_list, "LAC"))
#lac_2022_df_clean = lac_2022_df_clean[lac_2022_df_clean['possession_team'] == 'CLE']
#lac_2022_df = lac_2022_df[lac_2022_df['Down'] != 4]
lac_off_df = lac_2022_df.copy()
lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",None
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC


In [7]:
def detect_value_changes(df, column_name):
    change_indices = []
    prev_value = None
    
    for index, row in df.iterrows():
        current_value = row[column_name]
        if prev_value is not None and current_value != prev_value:
            change_indices.append(index)
        prev_value = current_value
    
    return change_indices

# Initialize 'team change' column with zeros
lac_off_df['team change'] = 0

# Update 'team change' with ones where there's a change in 'OPP' column
changes = detect_value_changes(lac_off_df, 'OPP')
for index in changes:
    lac_off_df.loc[index, 'team change'] = 1

lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,team change
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",None,0
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,0
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC,0
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,0


In [8]:
def update_rush_columns(row):
    detail = row['Detail']
    match_words = ['middle for', 'tackle for', 'guard for', 'end for']

    for word in match_words:
        if word in detail:
            yards_match = re.search(r'for\s+(\d+)\s+yards', detail)
            if yards_match:
                row['rush_play'] = 1
                row['rush_length'] = int(yards_match.group(1))
            else:
                row['rush_play'] = 0
                row['rush_length'] = 0
            return row
    
    row['rush_play'] = 0
    row['rush_length'] = 0
    return row

def update_rush_yards_tally(row, previous_tally, reset_tally):
    if reset_tally:
        row['rush_yards_tally'] = 0
    else:
        if row['rush_play'] == 1:
            row['rush_yards_tally'] = previous_tally + row['rush_length']
        else:
            row['rush_yards_tally'] = previous_tally
    return row

# Apply the function to each row to update rush columns
lac_off_df = lac_off_df.apply(lambda row: update_rush_columns(row), axis=1)

# Calculate cumulative rushing yards
cumulative_rush_yards = 0
reset_tally = False  # Variable to track if tally needs to be reset
lac_off_df['rush_yards_tally'] = 0
for index, row in lac_off_df.iterrows():
    lac_off_df.loc[index] = update_rush_yards_tally(row, cumulative_rush_yards, reset_tally)
    if row['team change'] == 1:  # Check if there's a team change
        reset_tally = True
    else:
        reset_tally = False
        cumulative_rush_yards = lac_off_df.loc[index, 'rush_yards_tally']

lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,team change,rush_play,rush_length,rush_yards_tally
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",None,0,0,0,0
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,0,1,8,8
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC,0,0,0,8
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0,1,2,10
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,0,1,2,12


In [9]:
def update_pass_columns(row):
    global previous_pass_tally  # Make previous_pass_tally accessible and modifiable within the function
    detail = row['Detail']
    if "pass incomplete" in detail:
        row['pass_complete'] = 0
        row['pass_incomplete'] = 1
        row['pass_play'] = 1
        row['pass_length'] = 0  # Set pass_length to 0 for incomplete passes
    elif "pass complete" in detail:
        yards_match = re.search(r'for\s+(\d+)\s+yards', detail)
        if yards_match:
            yards = int(yards_match.group(1))
            row['pass_complete'] = 1
            row['pass_incomplete'] = 0
            row['pass_play'] = 1
            row['pass_length'] = yards
            previous_pass_tally += yards  # Update pass yards tally
        else:
            row['pass_complete'] = 1
            row['pass_incomplete'] = 0
            row['pass_play'] = 1
            row['pass_length'] = 0  # Set pass_length to 0 if yards_match is None
    else:
        row['pass_complete'] = 0
        row['pass_incomplete'] = 0
        row['pass_play'] = 0
        row['pass_length'] = 0
    
    row['pass_yards_tally'] = previous_pass_tally  # Update pass yards tally
    return row

# Initialize cumulative passing yards and a flag to indicate team change
previous_pass_tally = 0
team_change_index = 0  # Initialize to a value that won't be present in the DataFrame

# Apply the function to each row to update pass columns
for index, row in lac_off_df.iterrows():
    if row['team change'] == 1:
        team_change_index = index
        previous_pass_tally = 0  # Reset passing yards tally to zero for team change
    else:
        if index > team_change_index:
            previous_pass_tally = lac_off_df.loc[team_change_index:index-1, 'pass_length'].sum()

    lac_off_df.loc[index] = update_pass_columns(row)

# Reset passing yards tally to zero for the very beginning of the frame
lac_off_df.loc[lac_off_df.index[0], 'pass_yards_tally'] = 0

lac_off_df.head()


KeyError: 'pass_length'

In [ ]:
#for index, row in lac_off_df.iterrows():
#    lac_off_df.at[index, 'play_yards'] = row['pass_length']

In [ ]:
# Set initial value for the first row
lac_off_df.at[lac_off_df.index[0], 'yardage_diff_1_2'] = lac_off_df.at[lac_off_df.index[0], 'ToGo']
lac_off_df.at[lac_off_df.index[0], 'yardage_diff_2_3'] = 0

# Iterate through the DataFrame to calculate 'yardage_diff_1_2'
for index, row in lac_off_df.iterrows():
    if row['Down'] == 1:  # If Down is 1, set 'yardage_diff_1_2' to "yards to go"
        lac_off_df.at[index, 'yardage_diff_1_2'] = row['ToGo']
    elif row['Down'] == 2:
        if index - 1 in lac_off_df.index:
            lac_off_df.at[index, 'yardage_diff_1_2'] = lac_off_df.at[index - 1, 'ToGo'] - row['ToGo']
        else:
            lac_off_df.at[index, 'yardage_diff_1_2'] = 0
    elif row['Down'] == 3:
        lac_off_df.at[index, 'yardage_diff_1_2'] = 0

# Set the value of 'yardage_diff_2_3' for all rows (since it remains constant)
lac_off_df['yardage_diff_2_3'] = 0
lac_off_df['yardage_diff_1_2'].describe()
lac_off_df.head()


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,team change,rush_play,rush_length,rush_yards_tally,pass_yards_tally,yardage_diff_1_2,yardage_diff_2_3
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",None,0,0,0,0,0.0,NaN,0
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,0,1,8,8,NaN,10.0,0
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC,0,0,0,8,NaN,8.0,0
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0,1,2,10,NaN,0.0,0
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,0,1,2,12,NaN,10.0,0


In [ ]:
lac_off_df['Quarter'] = lac_off_df['Quarter'].replace('OT', int('5'))
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,team change,rush_play,rush_length,rush_yards_tally,pass_yards_tally,yardage_diff_1_2,yardage_diff_2_3
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",None,0,0,0,0,0.0,NaN,0
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,0,1,8,8,NaN,10.0,0
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC,0,0,0,8,NaN,8.0,0
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0,1,2,10,NaN,0.0,0
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,0,1,2,12,NaN,10.0,0


In [ ]:
for index, row in lac_off_df.iterrows():
    if index > 2 and row['Down'] == 3:  # Exclude the first two rows and only calculate for Down == 3
        if (index - 1) in lac_off_df.index:  # Check if the previous index exists
            lac_off_df.at[index, 'yardage_diff_2_3'] = lac_off_df.at[index - 1, 'yardage_diff_1_2'] - row['yardage_diff_1_2']
        else:
            lac_off_df.at[index, 'yardage_diff_2_3'] = 0  # Set to 0 if the previous index doesn't exist
    else:
        lac_off_df.at[index, 'yardage_diff_2_3'] = 0  # Set to 0 for other rows or when Down != 3

(lac_off_df['yardage_diff_2_3'] != 0.0).describe()
#lac_off_df['yardage_diff_2_3'].describe()
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,possession_team,team change,rush_play,rush_length,rush_yards_tally,pass_yards_tally,yardage_diff_1_2,yardage_diff_2_3
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",None,0,0,0,0,0.0,NaN,0.0
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,LAC,0,1,8,8,NaN,10.0,0.0
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,LAC,0,0,0,8,NaN,8.0,0.0
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,LAC,0,1,2,10,NaN,0.0,8.0
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,LAC,0,1,2,12,NaN,10.0,0.0


In [ ]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bintercepted\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'intercepted'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'intercepted'] = 0
lac_off_df['intercepted'] 
intercepton_count = (lac_off_df['intercepted'] > 0).sum()
print(f"interception count: {intercepton_count}")
#lac_off_df.head()

interception count: 33


In [ ]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bfumbles\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'fumble'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'fumble'] = 0
fumble_count = (lac_off_df['fumble'] > 0).sum()
print(f"fumble count: {fumble_count}")

fumble count: 31


In [ ]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bpunts\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'punt'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'punt'] = 0
punt_count = (lac_off_df['punt'] > 0).sum()
print(f"punt count: {punt_count}")
#lac_off_df.head()

punt count: 147


In [ ]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bfield goal\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'field goal att'] = 1  # Set 'sacked' to 1 if 'sacked' is found
    else:
        lac_off_df.at[index, 'field goal att'] = 0
field_goal_count = (lac_off_df['field goal att'] > 0).sum()
print(f"field goal count: {field_goal_count}")
#lac_off_df.head()

field goal count: 73


In [ ]:
for index, row in lac_off_df.iterrows():
    play_detail = row['Detail']
    if re.search(r'\bsacked\b', play_detail):  # Check for 'sacked' in the play detail
        lac_off_df.at[index, 'sacked'] = 1  # Set 'sacked' to 1 if 'sacked' is found
        # Extract sack yards using regex
        sack_yards_match = re.search(r'(-?\d+)\s+yards', play_detail)  # Match negative or positive numbers
        if sack_yards_match:
            sack_yards = int(sack_yards_match.group(1))  # Extract the sack yards value
            lac_off_df.at[index, 'sack_yards'] = sack_yards  # Assign sack yards to the corresponding row
        else:
            lac_off_df.at[index, 'sack_yards'] = 0  # If no sack yards found, set it to 0
    else:
        lac_off_df.at[index, 'sacked'] = 0
        lac_off_df.at[index, 'sack_yards'] = 0
        
#lac_off_df.head()

for index, row in lac_off_df.iterrows():
    if index == 0:  
        lac_off_df.at[index, 'time_between_plays'] = 0  # or NaN
    elif row['Down'] > 1:
        if index - 1 in lac_off_df.index:
            lac_off_df.at[index, 'yardage_diff'] = abs(row['ToGo'] - lac_off_df.at[index - 1, 'ToGo'])
        else:
            lac_off_df.at[index, 'yardage_diff'] = 0  # or NaN
    elif row['Down'] == 1:
        lac_off_df.at[index, 'yardage_diff'] = row['ToGo']
lac_off_df.head()

In [ ]:
def yards_to_promised_land(location):
    if isinstance(location, str):  # Check if location is a string
        location_parts = location.split()
        if len(location_parts) >= 2:  # Ensure there are at least two parts
            return int(location_parts[1])
    return None  # Return None for non-string values or invalid strings
lac_off_df['yards_to_endzone'] = lac_off_df['Location'].apply(yards_to_promised_land)

lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,pass_yards_tally,yardage_diff_1_2,yardage_diff_2_3,intercepted,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,NaN,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,NaN,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,NaN,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,NaN,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0


create new column "conversion_successful" if  while on third down the resultmof the play is a rush length or pass length is greater than 'yards to go', then put 1 in the conversion seuccesfful 

In [ ]:
def update_1d_conversion_successful(row):
    if row['Down'] == 1:
        gain = max(row['rush_length'], row['pass_length'])  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
lac_off_df['1D_conv_good'] = lac_off_df.apply(update_1d_conversion_successful, axis=1)
conv_1D_good_count = (lac_off_df['1D_conv_good']).sum()
print(conv_1D_good_count)
lac_off_df.head()


KeyError: 'pass_length'

In [ ]:
def update_2d_conversion_successful(row):
    if row['Down'] == 2:
        gain = max(row['rush_length'], row['pass_length'])  # Calculate the gain based on rushing or passing
        if gain >= row['ToGo']:
            return 1
    return 0

# Apply the function to each row to update the conversion_successful column
lac_off_df['2D_conv_good'] = lac_off_df.apply(update_2d_conversion_successful, axis=1)
conv_2D_good_count = (lac_off_df['2D_conv_good']).sum()
print(conv_2D_good_count)
lac_off_df.head()

232


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,yardage_diff_2_3,intercepted,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone,1D_conv_good,2D_conv_good
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0,0
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0,0
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,0,0
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,0,0
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0,0


In [ ]:
def update_3d_conversion_successful(row):
    if row['Down'] == 3:  # Check if it's third down
        if row['rush_play'] == 1:  # If it's a rush play
            if row['rush_length'] >= row['ToGo']:  # Check if rushing yards are greater than or equal to yards to go
                return 1
        elif row['pass_play'] == 1:  # If it's a pass play
            if row['pass_length'] >= row['ToGo']:  # Check if pass length is greater than or equal to yards to go
                return 1
    return 0
# Apply the function to each row to update the conversion_successful column
lac_off_df['3D_conv_good'] = lac_off_df.apply(update_3d_conversion_successful, axis=1)

conv_3D_good_count = (lac_off_df['3D_conv_good']).sum()
print(conv_3D_good_count)

lac_off_df.head()


188


,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,intercepted,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone,1D_conv_good,2D_conv_good,3D_conv_good
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",...,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0,0,0
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0,0,0
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,0.0,0.0,0.0,0.0,0.0,33.0,0,0,0
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0.0,0.0,0.0,0.0,0.0,0.0,34.0,0,0,1
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0.0,0.0,0.0,0.0,0.0,36.0,0,0,0


In [ ]:
def update_4d_conversion_successful(row):
    if row['Down'] == 4:  # Check if it's third down
        if row['rush_play'] == 1:  # If it's a rush play
            if row['rush_length'] >= row['ToGo']:  # Check if rushing yards are greater than or equal to yards to go
                return 1
        elif row['pass_play'] == 1:  # If it's a pass play
            if row['pass_length'] >= row['ToGo']:  # Check if pass length is greater than or equal to yards to go
                return 1
    return 0
# Apply the function to each row to update the conversion_successful column
lac_off_df['4D_conv_good'] = lac_off_df.apply(update_4d_conversion_successful, axis=1)
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone,1D_conv_good,2D_conv_good,3D_conv_good,4D_conv_good
0,1,1,15:00,NaN,NaN,LVR 35,LVR,0.0,0.0,"Daniel Carlson kicks off 65 yards, touchback.",...,0.0,0.0,0.0,0.0,0.0,35.0,0,0,0,0
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0.0,0.0,0.0,0.0,25.0,0,0,0,0
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,0.0,0.0,0.0,0.0,33.0,0,0,0,0
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0.0,0.0,0.0,0.0,0.0,34.0,0,0,1,0
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0.0,0.0,0.0,0.0,36.0,0,0,0,0


In [ ]:
lac_off_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3063 entries, 0 to 3062
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   week              3063 non-null   int64  
 1   Quarter           3062 non-null   object 
 2   Time              3062 non-null   object 
 3   Down              2660 non-null   float64
 4   ToGo              2660 non-null   float64
 5   Location          2941 non-null   object 
 6   OPP               3063 non-null   object 
 7   OPP Score         2940 non-null   float64
 8   LAC Score         2940 non-null   float64
 9   Detail            3063 non-null   object 
 10  possession_team   1513 non-null   object 
 11  team change       3063 non-null   int64  
 12  rush_play         3063 non-null   int64  
 13  rush_length       3063 non-null   int64  
 14  pass_complete     3063 non-null   int64  
 15  pass_incomplete   3063 non-null   int64  
 16  pass_play         3063 non-null   int64  


In [ ]:
lac_off_df = lac_off_df[(lac_off_df['Quarter'] != "OT")]
lac_off_df = lac_off_df.dropna()
#lac_off_df = lac_off_df.astype({'Quarter': 'int8', 'Down': 'int8','ToGo': 'int8','Down': 'int8','Location':'string','OPP':'string','OPP Score':'int8','Detail':'string'})
#'Time':'string'
#lac_off_df_clean=lac_off_df.copy()
lac_off_df.head()

,week,Quarter,Time,Down,ToGo,Location,OPP,OPP Score,LAC Score,Detail,...,fumble,punt,field goal att,sacked,sack_yards,yards_to_endzone,1D_conv_good,2D_conv_good,3D_conv_good,4D_conv_good
1,1,1,15:00,1.0,10.0,LAC 25,LVR,0.0,0.0,Austin Ekeler left tackle for 8 yards (tackle ...,...,0.0,0.0,0.0,0.0,0.0,25.0,0,0,0,0
2,1,1,14:45,2.0,2.0,LAC 33,LVR,0.0,0.0,Justin Herbert pass complete short right to Ke...,...,0.0,0.0,0.0,0.0,0.0,33.0,0,0,0,0
3,1,1,14:07,3.0,1.0,LAC 34,LVR,0.0,0.0,Zander Horvath up the middle for 2 yards (tack...,...,0.0,0.0,0.0,0.0,0.0,34.0,0,0,1,0
4,1,1,13:24,1.0,10.0,LAC 36,LVR,0.0,0.0,Austin Ekeler right tackle for 2 yards (tackle...,...,0.0,0.0,0.0,0.0,0.0,36.0,0,0,0,0
5,1,1,12:50,2.0,8.0,LAC 38,LVR,0.0,0.0,Justin Herbert pass complete short left to Jos...,...,0.0,0.0,0.0,0.0,0.0,38.0,0,1,0,0


In [ ]:
lac_off_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1192 entries, 1 to 3048
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   week              1192 non-null   int64  
 1   Quarter           1192 non-null   object 
 2   Time              1192 non-null   object 
 3   Down              1192 non-null   float64
 4   ToGo              1192 non-null   float64
 5   Location          1192 non-null   object 
 6   OPP               1192 non-null   object 
 7   OPP Score         1192 non-null   float64
 8   LAC Score         1192 non-null   float64
 9   Detail            1192 non-null   object 
 10  possession_team   1192 non-null   object 
 11  team change       1192 non-null   int64  
 12  rush_play         1192 non-null   int64  
 13  rush_length       1192 non-null   int64  
 14  pass_complete     1192 non-null   int64  
 15  pass_incomplete   1192 non-null   int64  
 16  pass_play         1192 non-null   int64  


In [ ]:
count_greater_than_zero = (lac_off_df['1D_conv_good']).sum()
print(count_greater_than_zero)

97


In [ ]:
lac_off_df['yardage_diff_1_2'].describe()
#print("Sum of the values in the column:", column_sum)

count    1192.000000
mean        5.262584
std         5.131422
min       -11.000000
25%         0.000000
50%         6.000000
75%        10.000000
max        25.000000
Name: yardage_diff_1_2, dtype: float64